# Παράδειγμα Χρήσης Εργαλείου AutoGen


## Εισαγωγή των Απαραίτητων Πακέτων


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Δημιουργία του Client

Σε αυτό το παράδειγμα, θα χρησιμοποιήσουμε [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) για πρόσβαση στο LLM.

Το `model` ορίζεται ως `gpt-4o-mini`. Δοκιμάστε να αλλάξετε το μοντέλο σε κάποιο άλλο διαθέσιμο στην αγορά του GitHub Models για να δείτε διαφορετικά αποτελέσματα.

Ως μια γρήγορη δοκιμή, θα εκτελέσουμε απλώς ένα απλό prompt - `Ποια είναι η πρωτεύουσα της Γαλλίας`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Ορισμός των Συναρτήσεων

Σε αυτό το παράδειγμα, θα δώσουμε στον πράκτορα πρόσβαση σε ένα εργαλείο που είναι μια συνάρτηση με μια λίστα διαθέσιμων προορισμών διακοπών και τη διαθεσιμότητά τους.

Μπορείτε να σκεφτείτε ότι αυτό θα ήταν ένα σενάριο όπου ένας ταξιδιωτικός πράκτορας θα μπορούσε να έχει πρόσβαση σε μια ταξιδιωτική βάση δεδομένων, για παράδειγμα.

Καθώς προχωράτε σε αυτό το δείγμα, μη διστάσετε να δοκιμάσετε να ορίσετε νέες συναρτήσεις και εργαλεία που μπορεί να καλέσει ο πράκτορας.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Ορισμός του Εργαλείου Συνάρτησης 
Για να χρησιμοποιήσει ο πράκτορας το `vacation_destinations` ως `FunctionTool`, πρέπει να το ορίσουμε ως τέτοιο. 

Θα παρέχουμε επίσης μια περιγραφή του εργαλείου, η οποία είναι χρήσιμη για τον πράκτορα ώστε να κατανοήσει για ποιο σκοπό χρησιμοποιείται το εργαλείο σε σχέση με την εργασία που έχει ζητήσει ο χρήστης.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Ορισμός του agent

Τώρα μπορούμε να δημιουργήσουμε τον agent στον παρακάτω κώδικα. Ορίζουμε το `system_message` για να δώσουμε οδηγίες στον agent σχετικά με το πώς να βοηθά τους χρήστες να βρίσκουν προορισμούς διακοπών.

Επίσης, ορίζουμε την παράμετρο `reflect_on_tool_use` σε true. Αυτό επιτρέπει τη χρήση του LLM για να λαμβάνει την απόκριση από την κλήση του εργαλείου και να στέλνει την απάντηση χρησιμοποιώντας φυσική γλώσσα.

Μπορείτε να ορίσετε την παράμετρο σε false για να δείτε τη διαφορά.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Εκτέλεση του Agent

Τώρα μπορούμε να εκτελέσουμε τον agent με το αρχικό μήνυμα χρήστη που ζητά να κάνει ένα ταξίδι στο Τόκιο.

Μπορείτε να αλλάξετε αυτόν τον προορισμό πόλης για να δείτε πώς ο agent ανταποκρίνεται στη διαθεσιμότητα της πόλης.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
